# Philodoptera Dataset Conversion

Convert cricket tracking data to movement-compatible poses.csv format and then to TrialTree.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path
from audioio import load_audio

from movement.io import load_poses
from movement.kinematics import compute_velocity, compute_speed, compute_acceleration

from ethograph.utils.io import TrialTree, set_media_attrs, minimal_basics
from ethograph.features.audio_features import get_envelope

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_folder = Path(r"C:\Users\aksel\Documents\Code\EthoGraph\data\Philodoptera")

poses_csv_path = data_folder / "poses.csv"
audio_path = data_folder / "audio.wav"
video_path = data_folder / "video.mp4"

fps = 240
scorer = "Akseli"

In [3]:

ds = load_poses.from_dlc_file(poses_csv_path, fps=fps)
ds["velocity"] = compute_velocity(ds.position)
ds["speed"] = compute_speed(ds.position)
ds["acceleration"] = compute_acceleration(ds.position)

In [4]:
audio, sr = load_audio(str(audio_path))


In [5]:
ds.attrs["audio_sr"] = sr

ds['individuals'] = ["Pholidoptera_littoralis_1"]
ds.attrs["Recording info"] = "Akseli Ilmanen: This cricket species Pholidoptera littoralis was recorded on a field trip in Slovenia in 2025. The video was recorded with an iPhone and the audio with an Zoom recorder. Synchronization may not be perfect."#



In [6]:

ds = set_media_attrs(
    ds,
    cameras=[video_path.name],
    mics=[audio_path.name],
    tracking=[poses_csv_path.name],
    tracking_prefix="dlc"
)

dt = minimal_basics(ds, label_sr=1000)

output_path = data_folder / "Philodoptera_labelsr1000.nc"
dt.to_netcdf(output_path)

2026-01-16 09:06:41.724 | WARNING  | xarray.core.utils:emit_user_level_warning:1268 - C:\Users\aksel\Documents\Code\EthoGraph\ethograph\utils\io.py:338: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  np.zeros((len(time_labels), ds.dims["individuals"])),



Extracted type_vars_dict: {'individuals': array(['Pholidoptera_littoralis_1'], dtype='<U25'), 'features': ['position', 'confidence', 'velocity', 'speed', 'acceleration'], 'cameras': array(['cam1'], dtype='<U4'), 'mics': array(['mic1'], dtype='<U4'), 'tracking': array(['dlc1'], dtype='<U4'), 'keypoints': array(['LeftWingTip', 'RightWingTip'], dtype='<U12'), 'trial_conditions': []}


In [7]:
len(ds.labels)

3651